In [21]:
import json
from pickle import FALSE
from sys import displayhook
import warnings
import networkx as nx
import pandas as pd
import numpy as np
import timeit
import Graph_identifier as gf
import matplotlib.pyplot as plt
from IPython.display import Image
import os
import ipywidgets as widgets
import requests
from pyvis.network import Network
warnings.filterwarnings('ignore')


**---------------------------------------------FUNCTION PART-------------------------------------------**


In [22]:
proxies = {
   'http': 'http://172.19.1.222:3128',
   'https': 'http://172.19.1.222:3128',
}

In [23]:
def gettenant_access_token():
    tokenurl="https://open.feishu.cn/open-apis/auth/v3/tenant_access_token/internal"
    headers={"Content-Type":"application/json"}
    data={
        "app_id":"cli_a1f5f330a5f9d00e",
        "app_secret":"QFzhtAJwEdPiUGF9c4tpKeub2TEKI3x8"

    }
    request=requests.post(url=tokenurl,headers=headers,json=data,proxies=proxies)
    response=json.loads(request.content)['tenant_access_token']
    return response

def sendmes(chat_id,messages):
    tenant_access_token=gettenant_access_token()
    sendurl="https://open.feishu.cn/open-apis/message/v4/batch_send?receive_id_type=user_id"
    headers={"Authorization":"Bearer %s"%tenant_access_token,"Content-Type":"application/json"}
    data ={
            "user_ids": [chat_id],
            "msg_type": "post",
            "content": {
                "post":{
                    "zh_cn": {
                        "title": 'dependency Task',
                        "content": messages
                    }
            }
        }
    }
    request=requests.post(url=sendurl,headers=headers,json=data,proxies=proxies)
    print(request.content)

Searching Function:

In [24]:
def find_source(data):

    distin_source=int(input("input 1 for FWsearch or 2 for BWsearch:"))

    if os.path.isfile(r"Forward.graphml") and os.path.isfile(r"Backward.graphml"):
        print('file check ok')
    else:
        sql_data=gf.get_important_task()
        gf.Path_identification(sql_data) 

    if distin_source==1:
        G=nx.read_graphml("Forward.graphml")
    elif distin_source==2:
        G=nx.read_graphml("Backward.graphml") 
    else:
        print('input error')

# name = input("Target task:")
    dependences_set=set()
    for x in data:
        dependences_set|=set(nx.single_source_dijkstra_path(G,x).keys())
    
    return [G,list(dependences_set)]

**-----------------------------------------------TERMINAL INPUT PART-------------------------------------------**

In [25]:
#import csv file as a input
#df_input=pd.read_csv('gpnew.csv',header=0)

In [26]:
#input a list 
#df_input=['SQOOP_L_TO_DMS_PROD_RETURN','SQOOP_L_TO_DMS_RETURN_ANALISIS','SQOOP_L_TO_DMS_ORDER','SQOOP_L_TO_DMS_PROD_LIST']

In [27]:
#input one task name
df_input=input('Query task: ')

In [28]:
match df_input:
    case pd.DataFrame():
        data_input=df_input[df_input.columns.values[0]].values.tolist()
    case list():
        data_input=df_input
    case str:
        data_input=[df_input]

result=find_source(data_input)

file check ok


**-----------------------------------------------Graph Visualisation-------------------------------------------**

In [29]:
H1 = nx.subgraph(result[0], result[1])
for x in result[1]:
    if x in data_input:
        #FF6C6C
        H1.nodes[x]['color']='#97C2FC'
    else:
        #7F7FFF
        H1.nodes[x]['color']='#FFFF00'

# net=Network(notebook=True,bgcolor='#f5f5f5', font_color='black')
net=Network(height='100%', width='100%', directed=False, bgcolor='#f5f5f5', font_color='black')
net.from_nx(H1)
net.show("Result.html")

In [32]:

#show result dataframe:
H_reverse=nx.reverse(H1, copy=False)
ttt=list(H_reverse.edges(result[1]))
df=pd.DataFrame(ttt)
df.columns=['source','target']
df.sort_values(by=['source','target'],inplace=True)
df.to_csv('dep_task.csv',index=False)
df

,source,target
7,L_TS_POST_ACCT_CHRG_ITEM_REL_D,T_TR_L_POST_ACCT_CHRG_ITEM_REL
119,L_TS_POST_CHRG_ITEM_D,T_TR_L_POST_CHRG_ITEM
41,L_TS_POST_CNTCT_MED_D,T_TR_L_POST_CNTCT_MED
13,L_TS_POST_COST_CTR_D,T_TR_L_POST_COST_CTR
88,L_TS_POST_CUST_STS_D,T_TR_L_POST_CUST_STS
...,...,...
28,T_TW_POST_USR_852_D,T_TW_POST_USR_D
117,T_TW_POST_USR_D,T_TW_POST_USR_EXT_D
118,T_TW_POST_USR_D,T_VW_TW_POST_USR_D
14,T_TW_POST_USR_EXT_D,T_VW_TW_POST_USR_D


**-----------------------------------------------Try messagese-------------------------------------------**

In [31]:
# receiver='PT4358'
# message_1=[]
# nl='\n'
# if result[1]:
#     message_1.append([{
#                     "tag": "text",
#                     "text":  f"{nl.join(result[1])}"
#                     }])
# sendmes(receiver,message_1)

b'{"code":0,"data":{"invalid_department_ids":[],"invalid_open_ids":[],"invalid_union_ids":[],"invalid_user_ids":[],"message_id":"bm-fbd5d351c84211d42432b03d416b7271"},"msg":"ok"}'
